In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import heapq
import sys
sys.path.insert(1, '../framework')
from racetrack import *
rt = RACETrack()

In [ ]:
df = pd.DataFrame({'fm':['a',  'a',  'a',  'a',  'b',  'b',  'b',  'c',  'c',  'd',  'd',  'd', 'd'],
                   'to':['b',  'c',  'd',  'b',  'a',  'b',  'c',  'a',  'b',  'c',  'a',  'b', 'd'],
                   'ct':[10,   20,   5,    1,    20,   3,    5,    10,   15,   5,    10,   50,  20]})
def test(df, num, params):
    print(f'{num}.old_pd', rt.dendrogramOrdering(df,                       *params))
    print(f'{num}.old_pl', rt.dendrogramOrdering(pl.DataFrame(df),         *params))
    print(f'{num}.hdb_pd', rt.dendrogramOrdering_HDBSCAN(df,               *params))
    print(f'{num}.hdb_pl', rt.dendrogramOrdering_HDBSCAN(pl.DataFrame(df), *params))
def testAll(df):
    params = ['fm', 'to', None, False]
    test(df,1,params)
    print()
    params = ['fm', 'to', 'ct', False]
    test(df,2,params)
    print()
    params = ['fm', 'to', 'ct', True] 
    test(df,3,params)
testAll(df)

In [ ]:
df = pd.DataFrame({'fm':['a',  'x',  'n',  'r',  'h', 'h'],
                   'to':['r',  'y',  'm',  'g',  'a', 'i'],
                   'ct':[20,   20,   20,    1 ,   1,   1]})
params = ['fm', 'to', 'ct', False]
# test(df,4,params)
rt.tile([rt.chordDiagram(df, relationships=[('fm','to')],count_by='ct', draw_labels=True, txt_h=16, use_hdbscan_for_dendrogram=True),
         rt.chordDiagram(df, relationships=[('fm','to')],count_by='ct', draw_labels=True, txt_h=16, use_hdbscan_for_dendrogram=False)])

In [ ]:
_heap_, _graph_ = rt.__dendrogramHelper_pandas__(df, 'fm', 'to', 'ct', False)
# iteratively merge the closest nodes together
_sep_ = '|||'
_connector_ = ' <-|-> '
_tree_ = {}
_merged_already_ = set()
while len(_heap_) > 0:
    _strength_, _fmto_ = heapq.heappop(_heap_)
    _fm_, _to_ = rt.__den_fromToStringParts__(_fmto_)
    if _fm_ != _to_ and _fm_ not in _merged_already_ and _to_ not in _merged_already_:
        _merged_already_.add(_fm_), _merged_already_.add(_to_)

        _new_ = rt.__den_mergedName__(_fm_, _to_)
        _tree_[_new_] = (_fm_, _to_)

        _graph_[_new_] = {}
        # Rewire for _fm_
        for x in _graph_[_fm_].keys():
            if x not in _graph_[_new_].keys():
                _graph_[_new_][x] = 0    
            _graph_[_new_][x] += _graph_[_fm_][x]
        # Rewire for _to_
        for x in _graph_[_to_].keys():
            if x not in _graph_[_new_].keys():
                _graph_[_new_][x] = 0
            _graph_[_new_][x] += _graph_[_to_][x]
        # Rewire the neighbors & add the new values to the heap
        for x in _graph_[_new_].keys():
            _graph_[x][_new_] = _graph_[_new_][x]
            heapq.heappush(_heap_,(_graph_[_new_][x], _new_ + _connector_ + x))
        # Remove the old nodes and their nbor connections
        for x in _graph_[_fm_]:
            _graph_[x].pop(_fm_)
        _graph_.pop(_fm_)
        for x in _graph_[_to_]:
            _graph_[x].pop(_to_)
        _graph_.pop(_to_)
_tree_